## Предобработка / чтение

In [ ]:
pip install etna

In [ ]:
pip install ruptures

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date
from datetime import datetime, timedelta
import os
import prophet
import holidays
# импортируем необходимую функцию 
from statsmodels.tsa.stattools import adfuller
#добавим нужную библиотеку
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.signal import find_peaks
from statsmodels.graphics import tsaplots

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
for i in os.listdir('/content/drive/MyDrive/Датасеты/'):
    !unrar e /content/drive/MyDrive/Датасеты/{i}

## Определение динамики бронирований рейса в разрезе классов бронирования по вылетевшим рейсам. (2017-2019 год)

In [ ]:
# получение данных из файла по бронированию 
def get_data(path,flt_num,class_code,dd):
  class_df = pd.read_csv(path, delimiter=';')
  class_df = class_df.loc[(class_df['FLT_NUM']==flt_num) &
                          (class_df['SEG_CLASS_CODE']==class_code) & (class_df["DD"]==dd) & (class_df.DTD!=-1)]
  class_df.DD = class_df.DD.apply(lambda x: datetime.strptime(x, '%d.%m.%Y').date())
  class_df.SDAT_S = class_df.SDAT_S.apply(lambda x: datetime.strptime(x, '%d.%m.%Y').date())
  return class_df

In [ ]:
# получение динамики бронирования по конкретному рейсу и классу на определенную дату
def get_booking(df):

  if len(df)==0:
    print("Бронирований по данному рейсу нет")
    return None

  df = df.sort_values(by=["SDAT_S"])
  df = df[["SDAT_S","PASS_BK"]]
  df = df.sort_values(by=["SDAT_S"])

  start_date = df.SDAT_S.min()
  end_date = df.SDAT_S.max()

  res = pd.date_range(
      min(start_date, end_date),
      max(start_date, end_date)
  ).strftime('%Y-%m-%d')
  df["SDAT_S"] = df["SDAT_S"].astype("datetime64")
  df_data = pd.DataFrame(columns=["SDAT_S"],data=res)
  df_data["SDAT_S"] = df_data.SDAT_S.astype("datetime64")
  df = df_data.merge(df,on="SDAT_S",how="left").fillna(method="bfill").replace(np.nan,0)

  df["SDAT_S"] = df["SDAT_S"].apply(lambda x: datetime.date(x))

  if df["PASS_BK"].sum()==0:
    print("Бронирований за указанный период нет")
    return None
  

  # df.groupby('SDAT_S').sum().plot(kind='bar', figsize=(13,8), linewidth=1.0)

  
  return df[['SDAT_S', 'PASS_BK']]

In [ ]:
# получение роста динамики бронирования и нормализация этих значений
def get_booking_dynamics(data):

  # в относительных значениях 
   
  # будет с заполнением пропусков 
  df = get_booking(data.sort_values(by=["SDAT_S"]))
  if df is None:
    return None

  df = df.reset_index(drop=True)
  df["delta_book"] = df["PASS_BK"].shift(1,fill_value = df["PASS_BK"][0])
  df["delta"] = [max(0,(x-y))   for x,y in zip(df["PASS_BK"],df["delta_book"])]
  df["delta"][0]  = df["delta_book"][0]
  
  df = df[["SDAT_S","delta"]]
  df = df.sort_values(by=["SDAT_S"])

  start_date = df.SDAT_S.min()
  end_date = df.SDAT_S.max()

  res = pd.date_range(
      min(start_date, end_date),
      max(start_date, end_date)
  ).strftime('%Y-%m-%d')
  df["SDAT_S"] = df["SDAT_S"].astype("datetime64")
  df_data = pd.DataFrame(columns=["SDAT_S"],data=res)
  df_data["SDAT_S"] = df_data.SDAT_S.astype("datetime64")

  df = df_data.merge(df,on="SDAT_S",how="left").replace(np.nan,0)

  df["SDAT_S"] = df["SDAT_S"].apply(lambda x: datetime.date(x))

  df=df[['SDAT_S', 'delta']]

  if df["delta"].sum()==0:
    print("Динамики бронирований нет")
    return None
  

  df["delta_freq"] = df["delta"]/df["delta"].sum()
  # df[["SDAT_S","delta_freq"]].groupby('SDAT_S').sum().plot(kind='bar', figsize=(13,8), linewidth=1.0)


  return df[['SDAT_S', 'delta_freq']]

In [ ]:
# получение роста динамики бронирования
def get_absolute_booking_dynamics(data):
  # в абсолютных значениях для других графиков
   
  df = get_booking(data.sort_values(by=["SDAT_S"]))

  if len(df)==0:
    print("Бронирований нет")
    return None

  df = df.reset_index(drop=True)
  df["delta_book"] = df["PASS_BK"].shift(1,fill_value = df["PASS_BK"][0])
  df["delta"] = [max(0,(x-y))   for x,y in zip(df["PASS_BK"],df["delta_book"])]
  df["delta"][0]  = df["delta_book"][0]
  
  df = df[["SDAT_S","delta"]]
  df = df.sort_values(by=["SDAT_S"])

  start_date = df.SDAT_S.min()
  end_date = df.SDAT_S.max()

  res = pd.date_range(
      min(start_date, end_date),
      max(start_date, end_date)
  ).strftime('%Y-%m-%d')
  df["SDAT_S"] = df["SDAT_S"].astype("datetime64")
  df_data = pd.DataFrame(columns=["SDAT_S"],data=res)
  df_data["SDAT_S"] = df_data.SDAT_S.astype("datetime64")
  df = df_data.merge(df,on="SDAT_S",how="left").replace(np.nan,0)

  df["SDAT_S"] = df["SDAT_S"].apply(lambda x: datetime.date(x))

  df=df[['SDAT_S', 'delta']]

  if df["delta"].sum()==0:
    print("Динамики бронирований нет")
    return None
  

  # df[["SDAT_S","delta_freq"]].groupby('SDAT_S').sum().plot(kind='bar', figsize=(13,8), linewidth=1.0)


  return df[['SDAT_S', 'delta']]

In [ ]:
#example
flight_num = 1125
booking_class = 'J'
dd = "04.03.2019"
class_df = get_data('/content/CLASS_012018.csv', flight_num, booking_class,dd)
class_df = class_df.append(get_data('/content/CLASS_022018.csv', flight_num, booking_class,dd))
class_df = class_df.append(get_data('/content/CLASS_032018.csv', flight_num, booking_class,dd))
class_df = class_df.append(get_data('/content/CLASS_042018.csv', flight_num, booking_class,dd))
class_df = class_df.append(get_data('/content/CLASS_052018.csv', flight_num, booking_class,dd))
class_df = class_df.append(get_data('/content/CLASS_062018.csv', flight_num, booking_class,dd))
class_df = class_df.append(get_data('/content/CLASS_072018.csv', flight_num, booking_class,dd))
class_df = class_df.append(get_data('/content/CLASS_082018.csv', flight_num, booking_class,dd))
class_df = class_df.append(get_data('/content/CLASS_092018.csv', flight_num, booking_class,dd))
class_df = class_df.append(get_data('/content/CLASS_102018.csv', flight_num, booking_class,dd))
class_df = class_df.append(get_data('/content/CLASS_112018.csv', flight_num, booking_class,dd))
class_df = class_df.append(get_data('/content/CLASS_122018.csv', flight_num, booking_class,dd))
class_df = class_df.append(get_data('/content/CLASS_012019.csv', flight_num, booking_class,dd))
class_df = class_df.append(get_data('/content/CLASS_022019.csv', flight_num, booking_class,dd))
class_df = class_df.append(get_data('/content/CLASS_032019.csv', flight_num, booking_class,dd))
class_df = class_df.append(get_data('/content/CLASS_042019.csv', flight_num, booking_class,dd))
class_df = class_df.append(get_data('/content/CLASS_052019.csv', flight_num, booking_class,dd))
class_df = class_df.append(get_data('/content/CLASS_062019.csv', flight_num, booking_class,dd))
class_df = class_df.append(get_data('/content/CLASS_072019.csv', flight_num, booking_class,dd))
class_df = class_df.append(get_data('/content/CLASS_082019.csv', flight_num, booking_class,dd))
class_df = class_df.append(get_data('/content/CLASS_092019.csv', flight_num, booking_class,dd))
class_df = class_df.append(get_data('/content/CLASS_102019.csv', flight_num, booking_class,dd))
class_df = class_df.append(get_data('/content/CLASS_112019.csv', flight_num, booking_class,dd))
class_df = class_df.append(get_data('/content/CLASS_122019.csv', flight_num, booking_class,dd))
result = get_booking(class_df)

## Определение сезонности спроса по классам бронирования, по вылетевшим рейсам. (2017-2019 год)

### Сезонность сброса (без динамики бронирования)

In [ ]:
#  получение даты для определения сезонности 
def get_data_to_season(path,flt_num,class_code):
  class_df = pd.read_csv(path, delimiter=';')
  class_df = class_df.loc[(class_df['FLT_NUM']==flt_num) &
                          (class_df['SEG_CLASS_CODE']==class_code) & (class_df.DD==class_df.SDAT_S) ]
  class_df.DD = class_df.DD.apply(lambda x: datetime.strptime(x, '%d.%m.%Y').date())
  class_df.SDAT_S = class_df.SDAT_S.apply(lambda x: datetime.strptime(x, '%d.%m.%Y').date())
  return class_df

In [ ]:
import ruptures as rpt
def check_season(df):
  if df is None:
    return None

  start_date = df.DD.min()
  end_date = df.DD.max()

  res = pd.date_range(
      min(start_date, end_date),
      max(start_date, end_date)
  ).strftime('%Y-%m-%d')

  df_data = pd.DataFrame(columns=["DD"],data=res)
  df_data["DD"] = df_data.DD.astype("datetime64")

  df = df[["DD","PASS_BK"]]
    
  df["DD"] = df["DD"].astype("datetime64")
  df= df_data.merge(df,on="DD",how="left").replace(np.nan,0)

  


  df_to_save = df.copy()
  df = df.set_index("DD")
  decomposition = seasonal_decompose(df['PASS_BK'],model="additive")


  df_to_save["month"] = df_to_save["DD"].apply(lambda x: x.month)


  
  


  df_to_save = df_to_save.reset_index(drop=True)
  #сделать пока основные сезоны (летний, весенний,зимний и осенний)
  df_to_save["season"] = df_to_save.month.apply(lambda x: "весна" if (x>=3 and x<=5) else ("лето" if (x>=6 and x<=8) else ("осень" if (x>=9 and x<=11) else "зима")))

  result = pd.DataFrame(columns=["PASS_BK"], data = decomposition.trend.values)
  result["PASS_BK"] = result["PASS_BK"].replace(np.nan,0)
  result["DD"] = decomposition.trend.index
 
  result["season"] = df_to_save["season"]
  

  # off-line change point detection
  model = "l1"  
  algo = rpt.Dynp(model=model, min_size=15, jump=5).fit(df_to_save["PASS_BK"].values)
  my_bkps = algo.predict(n_bkps=10)
  rpt.show.display(df_to_save["PASS_BK"].values, my_bkps, figsize=(10, 6))
  plt.title('Change Point Detection: Dynamic Programming Search Method')
  plt.show()
    
  
  # прикрутить этот индекс к данным



  for i in range(1,len(my_bkps)-1):
    if my_bkps[i]!=0:
      if result.loc[my_bkps[i]:my_bkps[i+1]].PASS_BK.mean() > result.loc[my_bkps[i-1]:my_bkps[i]].PASS_BK.mean():
        # если среднее по этому промежутку больше, чем в прошлом, то он повышенный
        if "пониженный" not in result.loc[result.index == my_bkps[i-1], "season"].values[0]:
          season = result.loc[result.index == my_bkps[i], "season"].values[0]
          result.loc[my_bkps[i]]["season"] = "повышенный " + season
          j=my_bkps[i]+1
          while j<len(result) and j<= my_bkps[i+1]:
              season = result.loc[result.index == j, "season"].values[0]
              result.loc[result.index == j, "season"] = "повышенный " + season 
              j+=1

      else:
        # пониженный или обычный
        if "повышенный" not in result.loc[result.index == my_bkps[i-1], "season"].values[0]:
          # делаем пониженный
          season = result.loc[result.index == my_bkps[i], "season"].values[0]
          result.loc[my_bkps[i]]["season"] = "пониженный " + season
          j=my_bkps[i]+1
          while  j<len(result) and j<= my_bkps[i+1]:
              season = result.loc[result.index == j, "season"].values[0]
              result.loc[result.index == j, "season"] = "пониженный " + season 
              j+=1

  # добавить праздники
  for pass_bk,data,season in result.values:
    if data.date() in holidays.Russia(years=[2017,2018,2019]).keys():
          if  holidays.Russia(years=[2017,2018,2019])[data.date()]== "New Year Holidays" or holidays.Russia(years=[2017,2018,2019])[data.date()]=="Orthodox Christmas Day":
              result.loc[result.DD == data, "season"] = "Новогодние праздники"
        
    lower_bound = result.PASS_BK.quantile(q=0.025)
    upper_bound = result.PASS_BK.quantile(q=0.975)  
    date_ejection = result[(result.PASS_BK < lower_bound) | (result.PASS_BK > upper_bound)].DD.values
    if pass_bk>upper_bound:
      if data.date() in holidays.Russia(years=[2017,2018,2019]).keys():
          if  holidays.Russia(years=[2017,2018,2019])[data.date()]!= "New Year Holidays" or holidays.Russia(years=[2017,2018,2019])[data.date()]!="Orthodox Christmas Day":
            result.loc[result.DD == data, "season"] = holidays.Russia(years=[2017,2018,2019])[data.date()]

  return result[["DD","PASS_BK","season"]]

  


In [ ]:
# выгрузка за все время по конкретному классу и номеру рейса
flight_num = 1125
booking_class = 'J'
class_df_season = get_data_to_season('/content/CLASS_012018.csv', flight_num, booking_class)
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_022018.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_032018.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_042018.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_052018.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_062018.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_072018.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_082018.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_092018.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_102018.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_112018.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_122018.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_012019.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_022019.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_032019.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_042019.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_052019.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_062019.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_072019.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_082019.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_092019.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_102019.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_112019.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_122019.csv', flight_num, booking_class))
res= check_season(class_df_season)

### Сезонность спроса (с учетом динамики бронирования)

In [ ]:
# для сезонности динамики бронирования
def get_data_to_seasonv2(path,flt_num,class_code):
  class_df = pd.read_csv(path, delimiter=';')
  class_df = class_df.loc[(class_df['FLT_NUM']==flt_num) &
                          (class_df['SEG_CLASS_CODE']==class_code) & (class_df.DTD!=-1) & (~((class_df.FCLCLD==1) & (class_df.DD==class_df.SDAT_S)))] 
  class_df.DD = class_df.DD.apply(lambda x: datetime.strptime(x, '%d.%m.%Y').date())
  class_df.SDAT_S = class_df.SDAT_S.apply(lambda x: datetime.strptime(x, '%d.%m.%Y').date())
  return class_df

In [ ]:
def check_season_booking_dynamics(df):
  if df is None:
    return None
  start_date = df.SDAT_S.min()
  end_date = df.SDAT_S.max()
  sum=0
  try:
    res = pd.date_range(
        min(start_date, end_date),
        max(start_date, end_date)
    ).strftime('%Y-%m-%d')
  except ValueError:
    return None
  df_data = pd.DataFrame(columns=["SDAT_S"],data=res)
  df_data["SDAT_S"] = df_data.SDAT_S.astype("datetime64")
  df_data["delta"] = 0
  for dd_i in df.DD.unique():
    data_dd_i = get_absolute_booking_dynamics(df[df.DD==dd_i])
    if data_dd_i is None:
      continue
    data_dd_i.SDAT_S = data_dd_i.SDAT_S.astype("datetime64")
    df_data = df_data.merge(data_dd_i,on="SDAT_S",how="left").replace(np.nan,0)
    
    df_data["delta"] = df_data["delta_x"] + df_data["delta_y"]
    
    df_data = df_data.drop(columns=["delta_x","delta_y"])
    
  df_data["SDAT_S"] = df_data.SDAT_S.astype("datetime64")

  df_to_save = df_data.copy()
  df_data = df_data.set_index("SDAT_S")
  decomposition = seasonal_decompose(df_data['delta'],model="additive")

  df_to_save["month"] = df_to_save["SDAT_S"].apply(lambda x: x.month)




  df_to_save = df_to_save.reset_index(drop=True)
  #сделать пока основные сезоны (летний, весенний,зимний и осенний)
  df_to_save["season"] = df_to_save.month.apply(lambda x: "весна" if (x>=3 and x<=5) else ("лето" if (x>=6 and x<=8) else ("осень" if (x>=9 and x<=11) else "зима")))

  result = pd.DataFrame(columns=["delta"], data = decomposition.trend.values)
  result["delta"] = result["delta"].replace(np.nan,0)
  result["SDAT_S"] = decomposition.trend.index
 
  result["season"] = df_to_save["season"]
  

  model = "l1"  
  algo = rpt.Dynp(model=model, min_size=15, jump=5).fit(df_to_save["delta"].values)
  my_bkps = algo.predict(n_bkps=10)
    
  
  # прикрутить этот индекс к данным

  for i in range(1,len(my_bkps)-1):
    if my_bkps[i]!=0:
      if result.loc[my_bkps[i]:my_bkps[i+1]].delta.mean() > result.loc[my_bkps[i-1]:my_bkps[i]].delta.mean():
        # если среднее по этому промежутку больше, чем в прошлом, то он повышенный
        if "пониженный" not in result.loc[result.index == my_bkps[i-1], "season"].values[0]:
          season = result.loc[result.index == my_bkps[i], "season"].values[0]
          result.loc[my_bkps[i]]["season"] = "повышенный " + season
          j=my_bkps[i]+1
          while j<len(result) and j<= my_bkps[i+1]:
              season = result.loc[result.index == j, "season"].values[0]
              result.loc[result.index == j, "season"] = "повышенный " + season 
              j+=1

      else:
        # пониженный или обычный
        if "повышенный" not in result.loc[result.index == my_bkps[i-1], "season"].values[0]:
          # делаем пониженный
          season = result.loc[result.index == my_bkps[i], "season"].values[0]
          result.loc[my_bkps[i]]["season"] = "пониженный " + season
          j=my_bkps[i]+1
          while  j<len(result) and j<= my_bkps[i+1]:
              season = result.loc[result.index == j, "season"].values[0]
              result.loc[result.index == j, "season"] = "пониженный " + season 
              j+=1

  # добавить праздники
  for delta,data,season in result[["delta","SDAT_S","season"]].values:
    if data.date() in holidays.Russia(years=[2017,2018,2019]).keys():
          if  holidays.Russia(years=[2017,2018,2019])[data.date()]== "New Year Holidays" or holidays.Russia(years=[2017,2018,2019])[data.date()]=="Orthodox Christmas Day":
              result.loc[result.SDAT_S == data, "season"] = "Новогодние праздники"
        
    lower_bound = result.delta.quantile(q=0.025)
    upper_bound = result.delta.quantile(q=0.975)  
    date_ejection = result[(result.delta < lower_bound) | (result.delta > upper_bound)].SDAT_S.values
    if delta>upper_bound:
      if data.date() in holidays.Russia(years=[2017,2018,2019]).keys():
          if  holidays.Russia(years=[2017,2018,2019])[data.date()]!= "New Year Holidays" or holidays.Russia(years=[2017,2018,2019])[data.date()]!="Orthodox Christmas Day":
            result.loc[result.SDAT_S == data, "season"] = holidays.Russia(years=[2017,2018,2019])[data.date()]



  
  return result


In [ ]:
# выгрузка за все время по конкретному классу и номеру рейса
flight_num = 1125
booking_class = 'J'
class_df_season = get_data_to_season('/content/CLASS_012018.csv', flight_num, booking_class)
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_022018.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_032018.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_042018.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_052018.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_062018.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_072018.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_082018.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_092018.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_102018.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_112018.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_122018.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_012019.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_022019.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_032019.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_042019.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_052019.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_062019.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_072019.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_082019.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_092019.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_102019.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_112019.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_season('/content/CLASS_122019.csv', flight_num, booking_class))
class_df_season

In [ ]:
res = check_season_booking_dynamics(class_df_season2)

## Определение профиля бронирования

In [ ]:
def get_data_to_profile(path,flt_num, class_code):
  class_df = pd.read_csv(path, delimiter=';')
  class_df = class_df.loc[(class_df['FLT_NUM']==flt_num) &
                          (class_df['SEG_CLASS_CODE']==class_code) & (class_df.DTD!=-1) ]
  class_df.DD = class_df.DD.apply(lambda x: datetime.strptime(x, '%d.%m.%Y').date())
  class_df.SDAT_S = class_df.SDAT_S.apply(lambda x: datetime.strptime(x, '%d.%m.%Y').date())
  return class_df

In [ ]:
def check_profile(data):
  if data is None:
    return None
  set_delta = set()
  
  l = {
    "отдых":[],
    "бизнес/командировки":[],
    "спонтанное":[],
    "заранее запланированное":[]
  } 
  try:
    start_date = data.SDAT_S.min()
    end_date = data.SDAT_S.max()


    res = pd.date_range(
        min(start_date, end_date),
        max(start_date, end_date)
    ).strftime('%Y-%m-%d')

    for dd_i in data.DD.unique():
      data_i = data[data.DD==dd_i]
      
      res_booking =  get_booking_dynamics(data_i)
      if res_booking is not  None :
        day_boom = res_booking[res_booking.delta_freq == res_booking["delta_freq"].values.max()]["SDAT_S"].values[0]

        delta_active = (dd_i-day_boom).days
        if delta_active>60:
          l["заранее запланированное"] += [dd_i]

        elif delta_active<=60 and delta_active>15:
          l["отдых"] +=[dd_i]
        elif delta_active>=5 and delta_active<=15:
          l["бизнес/командировки"] +=[dd_i]
        else:
          l["спонтанное"] +=[dd_i]
    
    result = dict()

    if len(l)==0:
      return None
    df_result = pd.DataFrame(columns=["SDAT_S"],data = res)
    for key,values in l.items():
      if len(values)==0:
        continue
      # для каждого спроспа считаем спрос бронирования
      df = pd.DataFrame(columns=["SDAT_S"],data=res)
      for dd_i in values:
        res_data = get_absolute_booking_dynamics(data[data.DD==dd_i])
        res_data["SDAT_S"] = res_data["SDAT_S"].astype("datetime64")
        df["SDAT_S"] = df.SDAT_S.astype("datetime64")

        

        df = df.merge(res_data,how="outer",on="SDAT_S",suffixes=("","_y"))
        df = df.replace({np.nan:0})
        df["SDAT_S"] = df["SDAT_S"].apply(lambda x: datetime.date(x))

        if "delta_y" in df.columns:
          df["delta"] = df["delta"] + df["delta_y"]
          df = df.drop(columns=["delta_y"])
          df = df.sort_values(by=["SDAT_S"])
      result[key] = df 
      print("here")
      df_result[key] =df.delta.values

    
    

    return df_result
  except ValueError:
    return None

In [ ]:
flight_num = 1120
booking_class = "C"
class_df_season = get_data_to_profile('/content/CLASS_012018.csv', flight_num, booking_class)
class_df_season = class_df_season.append(get_data_to_profile('/content/CLASS_022018.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_profile('/content/CLASS_032018.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_profile('/content/CLASS_042018.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_profile('/content/CLASS_052018.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_profile('/content/CLASS_062018.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_profile('/content/CLASS_072018.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_profile('/content/CLASS_082018.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_profile('/content/CLASS_092018.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_profile('/content/CLASS_102018.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_profile('/content/CLASS_112018.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_profile('/content/CLASS_122018.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_profile('/content/CLASS_012019.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_profile('/content/CLASS_022019.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_profile('/content/CLASS_032019.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_profile('/content/CLASS_042019.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_profile('/content/CLASS_052019.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_profile('/content/CLASS_062019.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_profile('/content/CLASS_072019.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_profile('/content/CLASS_082019.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_profile('/content/CLASS_092019.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_profile('/content/CLASS_102019.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_profile('/content/CLASS_112019.csv', flight_num, booking_class))
class_df_season = class_df_season.append(get_data_to_profile('/content/CLASS_122019.csv', flight_num, booking_class))
res_delta = check_profile(class_df_season)